In [2]:
import requests as rq

url = 'https://presupuestoabierto.gob.cl/data/pagos.gz'
r = rq.get(url)

open('pagos.gz', 'wb').write(r.content)

1207404841

In [3]:
import gzip, shutil

with gzip.open('pagos.gz', 'r') as f_in, open('pagos.csv', 'wb') as f_out:
  shutil.copyfileobj(f_in, f_out)

In [4]:
import pandas as pd
df = pd.read_csv('pagos.csv', sep='\t')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6,10,11,12,13,14,15,16,17,18,26,33,35,36,37,39,41,43) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
#Imports necesarios
import time #librería necesaria para medir el tiempo de ejecución 
import datetime
import pandas as pd
import numpy as np
from functools import reduce
import urllib.request
import json

# Creamos los dataframes vacios para almacenar los valores de UTM y Fechas
dolarDiarioDF = pd.DataFrame(columns=['Fecha', 'Dolar'])
CLFDiarioDF = pd.DataFrame(columns=['Fecha', 'CLF'])
euroDiarioDF = pd.DataFrame(columns=['Fecha', 'Euro'])
utmDF = pd.DataFrame(columns=['Fecha', 'UTM'])
ipcDF = pd.DataFrame(columns=['Fecha', 'IPC'])

# Generamos el ciclo para recorrer los años y leer desde la API los datos de UTM
for i in range(2008, 2021):
    with urllib.request.urlopen("https://mindicador.cl/api/dolar/"+str(i)) as url:
        dolar = json.loads(url.read().decode())
    
    with urllib.request.urlopen("https://mindicador.cl/api/uf/"+str(i)) as url:
        uf = json.loads(url.read().decode())
    
    with urllib.request.urlopen("https://mindicador.cl/api/euro/"+str(i)) as url:
        euro = json.loads(url.read().decode())
    
    with urllib.request.urlopen("https://mindicador.cl/api/utm/"+str(i)) as url:
        utm = json.loads(url.read().decode())
        
    with urllib.request.urlopen("https://mindicador.cl/api/ipc/"+str(i)) as url:
        ipc = json.loads(url.read().decode())
    
    # Generamos los ciclos en que se añaden los dataframes los valores obtenidos desde la API    
    for x in range(len(dolar['serie'])):
        dolarDiarioDF = dolarDiarioDF.append({'Fecha' : dolar['serie'][x]['fecha'][0:7] , 'Dolar' : dolar['serie'][x]['valor']} , ignore_index=True) 
    
    for x in range(len(uf['serie'])):
        CLFDiarioDF = CLFDiarioDF.append({'Fecha' : uf['serie'][x]['fecha'][0:7] , 'CLF' : uf['serie'][x]['valor']} , ignore_index=True)
    
    for x in range(len(euro['serie'])):
        euroDiarioDF = euroDiarioDF.append({'Fecha' : euro['serie'][x]['fecha'][0:7] , 'Euro' : euro['serie'][x]['valor']} , ignore_index=True)
    
    for x in range(len(utm['serie'])):
        utmDF = utmDF.append({'Fecha' : utm['serie'][x]['fecha'][0:7] , 'UTM' : utm['serie'][x]['valor']} , ignore_index=True)
        
    for x in range(len(ipc['serie'])):
        ipcDF = ipcDF.append({'Fecha' : ipc['serie'][x]['fecha'][0:7] , 'IPC' : ipc['serie'][x]['valor']} , ignore_index=True)


# Agrupamos por promedio para efectos prácticos
dolarDF = dolarDiarioDF.groupby(['Fecha'])[['Dolar']].mean()
CLFDF = CLFDiarioDF.groupby(['Fecha'])[['CLF']].mean()
euroDF = euroDiarioDF.groupby(['Fecha'])[['Euro']].mean()

# Unimos los dataframes y lo guardamos en uno nuevo
monedas = reduce(lambda x,y: pd.merge(x,y, on='Fecha', how='left'), [dolarDF, CLFDF, euroDF, utmDF, ipcDF])

# calculamos el IPC en términos de su valor en índice
monedas['IPC'] = monedas['IPC'].fillna(0)
monedas['ipcIndices'] = 0.0
for index, row in monedas.iterrows():
    if index == 0:
        monedas['ipcIndices'][index] =  72.0007284469403   
    else:
        monedas['ipcIndices'][index] = monedas['ipcIndices'][index - 1] + monedas['IPC'][index] * monedas['ipcIndices'][index - 1] / 100

# en base al valor índice calculamos los factores de corrección y limpiamos el dataframe
monedas['factorIPC'] = 0.0
for index, row in monedas.iterrows():
    monedas['factorIPC'][index] = monedas['ipcIndices'][len(monedas)-1] / monedas['ipcIndices'][index]
    
monedas = monedas.drop(columns = ['IPC'])
#elimación de datos de IPC innecesarios
monedas = monedas.drop(columns = {'ipcIndices'})

In [9]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [10]:
#creación del campo mes como "numero" para hacer merge con monedas
df.loc[df['mes']==1, 'mesN'] = '01'
df.loc[df['mes']==2, 'mesN'] = '02'
df.loc[df['mes']==3, 'mesN'] = '03'
df.loc[df['mes']==4, 'mesN'] = '04'
df.loc[df['mes']==5, 'mesN'] = '05'
df.loc[df['mes']==6, 'mesN'] = '06'
df.loc[df['mes']==7, 'mesN'] = '07'
df.loc[df['mes']==8, 'mesN'] = '08'
df.loc[df['mes']==9, 'mesN'] = '09'
df.loc[df['mes']==10, 'mesN'] = '10'
df.loc[df['mes']==11, 'mesN'] = '11'
df.loc[df['mes']==12, 'mesN'] = '12'

In [11]:
#creación del campo año mes para fusionar con monedas
df['Fecha'] = df['periodo'].astype(str) + '-' + df['mesN']

In [12]:
# merge con monedas
df = df.merge(monedas, how = 'left', left_on = 'Fecha', right_on = 'Fecha')

In [13]:
# Calculo de devengo en pesos reales
df['DevengoReal'] = df['devengo']
df.loc[df['moneda']=='PESOS', 'DevengoReal'] = df['devengo'] * df['factorIPC']
df.loc[df['moneda']=='DOLARES', 'DevengoReal'] = df['devengo'] * df['factorIPC']

In [14]:
df['fecha_documento'] = pd.to_datetime(df['fecha_documento'], errors='coerce')
df['fecha_ingreso'] = pd.to_datetime(df['fecha_ingreso'], errors='coerce')
df['fecha_recepcion_conforme'] = pd.to_datetime(df['fecha_recepcion_conforme'], errors='coerce')
df['fecha_pago'] = pd.to_datetime(df['fecha_pago'], errors='coerce')
df['fecha_pago'] = pd.to_datetime(df['fecha_pago'], errors='coerce')
df['DevengoReal'] = round(df['DevengoReal']).astype('Int64')

In [26]:
df.keys()

Index(['periodo', 'mes', 'partida', 'nombre_partida', 'capitulo', 'nombre_capitulo', 'area', 'nombre_area', 'subtitulo', 'nombre_subtitulo', 'beneficiario', 'nombre_beneficiario', 'numero_documento', 'fecha_documento', 'tipo_documento', 'orden_de_compra', 'fecha_ingreso', 'fecha_recepcion_conforme', 'fecha_pago', 'moneda', 'monto_original', 'monto', 'devengo_original', 'devengo', 'honorario', 'proveedor', 'sector', 'region', 'agregado', 'persona', 'intraestado', 'bloqueo_oc', 'item', 'nombre_item', 'asignacion', 'nombre_asignacion', 'codigo_bip', 'nombre_bip', 'codigo_ubicacion_geografica', 'nombre_ubicacion_geografica', 'codigo_programa_presupuestario', 'nombre_programa_presupuestario', 'dias_de_pago', 'dias_de_pago_cat', 'deuda_flotante', 'mesN', 'Fecha', 'Dolar', 'CLF', 'Euro', 'UTM', 'factorIPC', 'DevengoReal'], dtype='object')

In [16]:
df2 = df[df['periodo']<=2020]

In [18]:
df2.to_csv('df_2020.csv', sep=';')